In [1]:
import unittest
import pandas as pd
from numpy.testing import assert_array_equal

from pint import set_application_registry
from pint_pandas import PintArray, PintType
from openscm_units import unit_registry
PintType.ureg = unit_registry
ureg = unit_registry
set_application_registry(ureg)
Q_ = ureg.Quantity
PA_ = PintArray

ureg.define("CO2e = CO2 = CO2eq = CO2_eq")

def _get_cumulative_emission(projected_emission_intensity, projected_production):
    df = projected_emission_intensity.multiply(projected_production)
    return df.sum(axis=1).astype('pint[Mt CO2]')

class TestBaseProvider(unittest.TestCase):
    """
    Test the Base provider
    """

    def setUp(self) -> None:
        pass
    
    def test_pint_series_equality(self):
        projected_ei = pd.DataFrame([[Q_(1.0, 't CO2/MWh'), Q_(2.0, 't CO2/MWh')], [Q_(3.0, 't CO2/MWh'), Q_(4.0, 't CO2/MWh')]], dtype='pint[t CO2/MWh]')
        projected_production = pd.DataFrame([[Q_(2.0, 'TWh'), Q_(4.0, 'TWh')], [Q_(6.0, 'TWh'), Q_(8.0, 'TWh')]], dtype='pint[TWh]')
        expected_data = pd.Series([10.0, 50.0],
                                    index=[0, 1],
                                    dtype='pint[Mt CO2]')
        print(_get_cumulative_emission(projected_emission_intensity=projected_ei,
                                       projected_production=projected_production))
        print(f"expected_data = {expected_data}")
        pd.testing.assert_series_equal(
            _get_cumulative_emission(projected_emission_intensity=projected_ei,
                                     projected_production=projected_production), expected_data)

x = TestBaseProvider('test_pint_series_equality')
TestBaseProvider.test_pint_series_equality(x)

0    10.000000000000002
1    50.000000000000014
dtype: pint[CO2 * megametric_ton]
expected_data = 0    10.0
1    50.0
dtype: pint[CO2 * megametric_ton]


/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


TypeError: object of type 'numpy.float64' has no len()